In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd
sys.path.append(str(Path(getcwd()).parent.parent)) ## ../../covid19datascience* <- set parent level here

In [ ]:
import pandas as pd
from epiweeks import Week
from datetime import timedelta, date
from os.path import join

from bulletin import default_input, default_output, root, hoje, data_comeco_pandemia
from bulletin.services.metabase import Metabase
from bulletin.systems.notifica import Notifica
from bulletin.utils.normalize import trim_overspace

In [ ]:
notifica = Notifica()
notifica.load('notifica', compress=False)

In [ ]:
# notifica.df.loc[notifica.df['etnia']!=-1].groupby(['raca_cor','etnia','classificacao_final'])[['id']].count().rename(index=notifica.tables['raca_cor'].set_index('id').to_dict()['raca_cor'],level='raca_cor').rename(index=notifica.tables['etnia'].set_index('co_etnia').to_dict()['etnia'],level='etnia').unstack('classificacao_final').fillna(0).astype(int).rename(columns=notifica.tables['classificacao_final'].set_index('id').to_dict()['classificacao_final'],level='classificacao_final').to_excel('raca_cor_etnia.xlsx')

In [ ]:
# notifica.df.loc[(notifica.df['etnia']!=-1) & (notifica.df['classificacao_final']==2)].groupby(['etnia',notifica.df.loc[notifica.df['etnia']!=-1,'data_notificacao'].rename('ano').dt.year])[['id']].count().rename(index=notifica.tables['etnia'].set_index('co_etnia').to_dict()['etnia'],level='etnia').unstack('ano').fillna(0).astype(int)

In [ ]:
# notifica.df.loc[notifica.df['etnia']!=-1,'raca_cor'] = 5
# notifica.df.loc[(notifica.df['etnia']==-1) & (notifica.df['raca_cor']==5),'raca_cor'] = 99

In [ ]:
indigenas = notifica.df.loc[(notifica.df['raca_cor']==5) & (notifica.df['excluir_ficha']==2) & (notifica.df['uf_residencia']==41)].copy()
# indigenas.groupby('etnia')[['id']].count().sort_values('id',ascending=False)

In [ ]:
# DOWNLOAD PPL
mb = Metabase()

ppl_path = join(default_input,'semanais','ppl.csv')
with open(join(mb.sql_path,'PPL.sql')) as file:
        raw_sql = trim_overspace(" ".join([ row.replace('\n',' ') for row in file.readlines() if not '--' in row ]))

mb.download(raw_sql, ppl_path)

In [ ]:
# ppl = notifica.df.loc[(notifica.df['ocupacao'].isin([5,0])) & (notifica.df['paciente_institucionalizado'].isin([4,1])) & (notifica.df['excluir_ficha']==2) & (notifica.df['uf_residencia']==41)].copy()
ppl = pd.read_csv(join(default_input,'semanais','ppl.csv'), converters = {
                            'data_notificacao' : pd.to_datetime,
                            },
                            sep=',',
                            encoding='utf-8-sig')
ppl.shape[0]

In [ ]:
sexo = notifica.tables['sexo'].copy()
raca_cor = notifica.tables['raca_cor'].copy()
etnia = notifica.tables['etnia'].copy()
classificacao_final = notifica.tables['classificacao_final'].copy()
evolucao = notifica.tables['evolucao'].copy()
paciente_institucionalizado = notifica.tables['paciente_institucionalizado'].copy()

In [ ]:
ppl = ppl[['data_notificacao', 'sexo', 'idade', 'uf_residencia', 'paciente_institucionalizado', 'classificacao_final', 'evolucao']]
ppl_menores = ppl.loc[ppl['idade'] < 10]
ppl = ppl.loc[ppl['idade'] >= 10]
ppl.loc[ppl['evolucao'] == 2, 'evolucao'] = 5
ppl.loc[(ppl['evolucao'] == 0) | (ppl['evolucao'].isnull()), 'evolucao'] = 3
ppl =  ppl.rename(columns={'sexo':'id'})
ppl = ppl.merge(how = 'left', on = 'id', right = sexo)
ppl = ppl[['data_notificacao', 'sexo', 'idade', 'uf_residencia', 'paciente_institucionalizado', 'classificacao_final', 'evolucao']]
ppl =  ppl.rename(columns={'paciente_institucionalizado':'id'})
ppl = ppl.merge(how = 'left', on = 'id', right = paciente_institucionalizado)
ppl = ppl[['data_notificacao', 'sexo', 'idade', 'uf_residencia', 'paciente_institucionalizado', 'classificacao_final', 'evolucao']]
ppl =  ppl.rename(columns={'classificacao_final':'id'})
ppl = ppl.merge(how = 'left', on = 'id', right = classificacao_final)
ppl = ppl[['data_notificacao', 'sexo', 'idade', 'uf_residencia', 'paciente_institucionalizado', 'classificacao_final', 'evolucao']]
ppl =  ppl.rename(columns={'evolucao':'id'})
ppl = ppl.merge(how = 'left', on = 'id', right = evolucao)
ppl = ppl[['data_notificacao', 'sexo', 'idade', 'uf_residencia', 'paciente_institucionalizado', 'classificacao_final', 'evolucao']]


tabela_cf = ppl.groupby(['classificacao_final'])[['data_notificacao']].count().sort_values('data_notificacao', ascending = False)
tabela_ev = ppl.groupby(['evolucao'])[['data_notificacao']].count().sort_values('data_notificacao', ascending = False)


ppl.to_csv(join(default_output, 'enviar_celepar','CORONAVIRUS.ppl.csv'), index = False, sep=';', encoding='utf-8-sig', date_format='%d/%m/%Y')
#ppl_menores.to_excel(join('ppl_menores.xlsx'), encoding='utf-8-sig')
tabela_cf.to_excel(join(default_output, 'semanais','TABELA_ppl_classificacao.xlsx'), encoding='utf-8-sig')
tabela_ev.to_excel(join(default_output, 'semanais','TABELA_ppl_evolucao.xlsx'), encoding='utf-8-sig')
ppl_menores.shape[0]

In [ ]:
indigenas = indigenas[['data_notificacao', 'raca_cor', 'sexo', 'idade', 'etnia', 'uf_residencia', 'classificacao_final', 'evolucao']]
indigenas.loc[indigenas['evolucao'] == 2, 'evolucao'] = 5
indigenas.loc[(indigenas['evolucao'] == 0) | (indigenas['evolucao'].isnull()), 'evolucao'] = 3
indigenas =  indigenas.rename(columns={'sexo':'id'})
indigenas = indigenas.merge(how = 'left', on = 'id', right = sexo)
indigenas = indigenas[['data_notificacao', 'raca_cor', 'sexo', 'idade', 'etnia', 'uf_residencia', 'classificacao_final', 'evolucao']]
indigenas =  indigenas.rename(columns={'raca_cor':'id'})
indigenas = indigenas.merge(how = 'left', on = 'id', right = raca_cor)
indigenas = indigenas[['data_notificacao', 'raca_cor', 'sexo', 'idade', 'etnia', 'uf_residencia', 'classificacao_final', 'evolucao']]
indigenas =  indigenas.rename(columns={'etnia':'co_etnia'})
indigenas = indigenas.merge(how = 'left', on = 'co_etnia', right = etnia)
indigenas = indigenas[['data_notificacao', 'raca_cor', 'sexo', 'idade', 'etnia', 'uf_residencia', 'classificacao_final', 'evolucao']]
indigenas =  indigenas.rename(columns={'classificacao_final':'id'})
indigenas = indigenas.merge(how = 'left', on = 'id', right = classificacao_final)
indigenas = indigenas[['data_notificacao', 'raca_cor', 'sexo', 'idade', 'etnia', 'uf_residencia', 'classificacao_final', 'evolucao']]
indigenas =  indigenas.rename(columns={'evolucao':'id'})
indigenas = indigenas.merge(how = 'left', on = 'id', right = evolucao)
indigenas = indigenas[['data_notificacao', 'raca_cor', 'sexo', 'idade', 'etnia', 'uf_residencia', 'classificacao_final', 'evolucao']]
indigenas.loc[indigenas['etnia'].isnull(), 'etnia'] = 'Não Informado'


tabela_cf = indigenas.groupby(['classificacao_final'])[['data_notificacao']].count().sort_values('data_notificacao', ascending = False)
tabela_et = indigenas.groupby(['etnia'])[['data_notificacao']].count().sort_values('data_notificacao', ascending = False)
tabela_ev = indigenas.groupby(['evolucao'])[['data_notificacao']].count().sort_values('data_notificacao', ascending = False)


# indigenas.to_csv(join(default_output,'enviar_celepar', 'CORONAVIRUS.indigenas.csv'), index = False, sep=';', encoding='utf-8-sig', date_format='%d/%m/%Y')
# tabela_cf.to_excel(join(default_output, 'semanais','TABELA_indigenas_classificacao.xlsx'), encoding='utf-8-sig')
# tabela_et.to_excel(join(default_output, 'semanais','TABELA_indigenas_etnias.xlsx'), encoding='utf-8-sig')
# tabela_ev.to_excel(join(default_output, 'semanais','TABELA_indigenas_evolucao.xlsx'), encoding='utf-8-sig')

In [ ]:
display(indigenas.groupby('classificacao_final')[['sexo']].count())